In [1]:
import numpy as np
import gustaf as gus
import splinepy as sp
import vedo

import os
os.chdir("/home/michael.kofler/DeepSDF")

vedo.settings.default_backend = 'k3d'

from sdf_sampler.plotting import scatter_contour_at_z_level
import matplotlib.pyplot as plt

import torch
from deep_sdf import workspace as ws
import deep_sdf.utils
import pathlib

import igl

params = {'text.usetex': False, 'mathtext.fontset': 'cm', 'axes.labelsize': 12}
plt.rcParams.update(params)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
experiment_directory = "experiments/double_lattice_3D"
checkpoint = "1000"

graded = True

latent = ws.load_latent_vectors(experiment_directory, checkpoint).to("cpu").numpy()
decoder = ws.load_trained_model(experiment_directory, checkpoint).to(device)
decoder.eval()
latent_base = np.array([0, -0.4])

In [3]:
control_points_ungraded = np.array([latent_base]*4)
control_points_graded = control_points_ungraded
# control_points_graded[3] += 0.2

tiling = [2, 2, 1]
N_base = 20


control_points_for_min_max = np.vstack([control_points_graded, control_points_ungraded])

if graded:
    graded_string = "_single_graded_derivative"
    control_points = np.vstack([control_points_graded, control_points_graded])
else:
    graded_string = "_single"
    control_points = np.vstack([control_points_ungraded, control_points_ungraded])

latent_vec_interpolation = sp.BSpline(
    degrees=[1, 1, 1],
    knot_vectors=[[-1, -1, 1, 1], 
                [-1, -1, 1, 1], 
                [-1, -1, 1, 1]],
    control_points=control_points,
)

In [4]:

def transform(x, t):
    p = 2/t
    return (2/p)*torch.abs((x-t%2) % (p*2) - p) -1 

def sdf_struct(queries):
    queries = torch.tensor(queries, dtype=torch.float32).to(device)
    tx, ty, tz = tiling


    samples = torch.zeros(queries.shape[0], 3)
    samples[:, 0] = transform(queries[:, 0], tx)
    samples[:, 1] = transform(queries[:, 1], ty)
    samples[:, 2] = transform(queries[:, 2], tz)
    lat_vec_red = torch.tensor(latent_vec_interpolation.evaluate(queries.cpu().numpy()), dtype=torch.float32)
    queries = torch.hstack([torch.tensor(lat_vec_red).to(torch.float32).to(device), samples])

    return deep_sdf.utils.decode_sdf(decoder, None, queries).squeeze(1).detach().cpu().numpy()


In [5]:
cap_border_dict = {
    "x0": {"cap": 1, "measure": 0.1},
    "x1": {"cap": 1, "measure": 0.1},
    "y0": {"cap": 1, "measure": 0.1},
    "y1": {"cap": 1, "measure": 0.1},
}

N = [N_base * t+1 for t in tiling]

verts, faces, jac = deep_sdf.mesh.create_mesh_microstructure_diff(tiling, decoder, latent_vec_interpolation, cap_border_dict=cap_border_dict, N=N, device=device, compute_derivatives=True)
jac = jac.reshape((jac.shape[0], jac.shape[1], -1))
verts_np = verts.detach().cpu().numpy()
faces_np = faces.detach().cpu().numpy()

sampling takes: 0.145322
sampling takes: 0.198087


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.74 GiB. GPU 0 has a total capacty of 47.43 GiB of which 3.75 GiB is free. Including non-PyTorch memory, this process has 43.67 GiB memory in use. Of the allocated memory 38.51 GiB is allocated by PyTorch, and 4.85 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [6]:
faces = []
jac[np.where(jac>1)] = 0
jac[np.where(jac<-1)] = 0
for i in range(jac.shape[2]):
    faces_der1 = gus.Faces(verts_np, faces_np)
    directions = jac
    positions = verts_np
    faces_der1.vertex_data["directions"] = jac[:,:,i]
    faces_der1.show_options["arrow_data"] = "directions"
    faces.append(faces_der1)
gus.show(*faces)


K3DPlotterN(children=(Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680…

In [7]:
print(jac.mean())

2.038772617134522e-05
